In [10]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [5]:
import gzip
import simplejson as json

In [6]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:


def describe(key):
    import statistics
    lst=[]
    for dic in scripts:
        for k, v in dic.items():
            if k == key:
                lst.append(v)
    lst=sorted(lst)    
    total = sum(lst)
    avg = statistics.mean(lst)
    s = statistics.stdev(lst)
    mid=len(lst)//2
    x=lst[:mid]
    y=lst[mid:]
    q25 = statistics.median(x)
    q75 = statistics.median(y)
    
    med = statistics.median(lst)
    

    return (total, avg, s, q25, med, q75)

print(describe("quantity"))

(316356836, 826.5883059943667, 3872.1860733044696, 30, 120.0, 466)


In [11]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]
summary

[('items', (4410054, 11.522744731217633, 33.11220959819492, 1, 3.0, 8)),
 ('quantity',
  (316356836, 826.5883059943667, 3872.1860733044696, 30, 120.0, 466)),
 ('nic',
  (29048309.790000085,
   75.89844899484227,
   197.57308474088757,
   7.7,
   22.62,
   65.94)),
 ('act_cost',
  (27053937.60000044,
   70.68748295124972,
   183.26755837715774,
   7.25,
   21.24,
   61.53))]

In [12]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [13]:
bnf_names = []  #making a list of all bnf_names
for i in scripts:
    if i["bnf_name"] not in bnf_names:
        bnf_names.append(i["bnf_name"])
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [15]:
groups = {name: [] for name in bnf_names}  #making a group which represents list of items in each bnf_name for the whole dataset

for script in scripts:
    
    groups[script["bnf_name"]].append(script["items"])
    
list(groups.items())[0]

('Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 [2,
  1,
  2,
  6,
  2,
  10,
  4,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  6,
  1,
  3,
  1,
  1,
  1,
  1,
  5,
  20,
  2,
  5,
  3,
  1])

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [16]:
total_items_bnf=[(k,sum(v)) for k,v in groups.items()] #calculating the total items in each bnf_names 

In [17]:
sorted(total_items_bnf)[:5]

[('3m Health Care_Cavilon Durable Barrier C', 1998),
 ('3m Health Care_Cavilon No Sting 1ml Barr', 247),
 ('3m Health Care_Cavilon No Sting 3ml Barr', 39),
 ('3m Health Care_Cavilon No Sting Barrier', 350),
 ('A.S Saliva Orthana Spy 50ml (App)', 25)]

In [18]:
max_item = [sorted(total_items_bnf,key=lambda x:x[1],reverse=True)[0]] #sorting the list in decending order to find out which bnf_names has maximum items
print(max_item)

[('Omeprazole_Cap E/C 20mg', 113826)]


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [19]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [20]:
def group_by_field(data, fields):  #making a fuction which can make a group like bnf_names for any fields in any data set
    
    for field in fields:
        names=[]  
        for i in data:
            if i[field] not in names:
                names.append(i[field])    #making a list of fields
            
        groups= {name: [] for name in names}
        for d in data:
    
            groups[d[field]].append(d["items"])  #making a list of items in each field
        
        
    
    return groups

In [21]:
groups = group_by_field(scripts, ('bnf_name',))
result=[(k,sum(v)) for k,v in groups.items()]
test_max_item = [sorted(result,key=lambda x:x[1],reverse=True)[0]]
print(test_max_item)

assert test_max_item == max_item

[('Omeprazole_Cap E/C 20mg', 113826)]


## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [22]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

In [23]:
practices[0]

{'code': 'A81001',
 'name': 'THE DENSHAM SURGERY',
 'addr_1': 'THE HEALTH CENTRE',
 'addr_2': 'LAWSON STREET',
 'borough': 'STOCKTON ON TEES',
 'village': 'CLEVELAND',
 'post_code': 'TS18 1HU'}

In [24]:
lst=[]
for practice in practices[:50]:
    lst.append((practice["code"],practice["post_code"]))
print(sorted(lst,key=lambda x:x[1]))

[('A81023', 'TS1 2NX'), ('A81029', 'TS1 2NX'), ('A81037', 'TS1 2NX'), ('A81004', 'TS1 3BE'), ('A81016', 'TS1 3QY'), ('A81035', 'TS1 3RX'), ('A81030', 'TS1 3RY'), ('A81045', 'TS10 1SR'), ('A81015', 'TS10 1TZ'), ('A81054', 'TS10 1UA'), ('A81018', 'TS10 4NW'), ('A81052', 'TS10 4NW'), ('A81047', 'TS11 6BW'), ('A81048', 'TS11 7BL'), ('A81013', 'TS12 2FF'), ('A81022', 'TS12 2TG'), ('A81053', 'TS13 4HW'), ('A81005', 'TS14 7DJ'), ('A81032', 'TS14 7DJ'), ('A81027', 'TS15 9DD'), ('A81039', 'TS16 9EA'), ('A81017', 'TS17 0EE'), ('A81034', 'TS17 0EE'), ('A81001', 'TS18 1HU'), ('A81025', 'TS18 1HU'), ('A81046', 'TS18 1YE'), ('A81006', 'TS18 2AT'), ('A81002', 'TS18 2AW'), ('A81036', 'TS20 2UZ'), ('A81040', 'TS23 2DG'), ('A81014', 'TS23 2LA'), ('A81007', 'TS24 7PW'), ('A81011', 'TS24 7PW'), ('A81031', 'TS24 7PW'), ('A81041', 'TS24 9DN'), ('A81044', 'TS25 1QU'), ('A81003', 'TS26 8DB'), ('A81012', 'TS3 6AL'), ('A81033', 'TS3 6AL'), ('A81038', 'TS3 6AL'), ('A81049', 'TS3 6AL'), ('A81019', 'TS3 7RL'), ('A

In [25]:
from collections import defaultdict
practice_postal = defaultdict(list)

for practice in practices:
        
    practice_postal[practice['code']].append(practice['post_code'])
        
practice_postal

defaultdict(list,
            {'A81001': ['TS18 1HU', 'TS18 1HU'],
             'A81002': ['TS18 2AW'],
             'A81003': ['TS26 8DB', 'TS25 1QU'],
             'A81004': ['TS1 3BE', 'TS5 8SB'],
             'A81005': ['TS14 7DJ'],
             'A81006': ['TS18 2AT', 'TS18 2AT'],
             'A81007': ['TS24 7PW'],
             'A81008': ['TS6 6TD', 'TS6 9QG'],
             'A81009': ['TS5 6HF'],
             'A81011': ['TS24 7PW'],
             'A81012': ['TS3 6AL'],
             'A81013': ['TS12 2FF'],
             'A81014': ['TS23 2LA'],
             'A81015': ['TS10 1TZ'],
             'A81016': ['TS1 3QY'],
             'A81017': ['TS17 0EE'],
             'A81018': ['TS10 4NW'],
             'A81019': ['TS3 7RL'],
             'A81020': ['TS4 3BU'],
             'A81021': ['TS6 6TD'],
             'A81022': ['TS12 2TG'],
             'A81023': ['TS1 2NX'],
             'A81025': ['TS18 1HU'],
             'A81026': ['TS5 6HA'],
             'A81027': ['TS15 9DD'],
         

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [26]:
def group_by_field(data, fields):
    
    for field in fields:
        names=[]
        for i in data:
            if i[field] not in names: #making a list of names in the given field
                names.append(i[field])
            
        groups= {name: [] for name in names}  
        for d in data:
    
            groups[d[field]].append(d["post_code"])  #making a group which represents a list of "Post_code" for each name of field
        
        for k,v in groups.items():
            groups[k]=sorted(v)[0]   #sorting the list in ascending order and keeping the first one
            
    
    return groups

In [27]:
practice_postal = group_by_field(practices, ('code',))  #making a group using "code" field in "practices " dataset

In [28]:
list(practice_postal.items())[:2]

[('A81001', 'TS18 1HU'), ('A81002', 'TS18 2AW')]

In [29]:
practice_postal['K82019']

'HP21 8TR'

In [30]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [31]:
joined = scripts[:]  #making a new variable copying the scripts dataset (shallow copy)
    

for script in joined:
    script['post_code'] =practice_postal[script["practice"]] #making a new field in the dataset named "post_code" in which the value comes from "practice" dataset

In [32]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

In [33]:
d={} #making a dictionary which will count the "items" in each "post_code"
for script in joined:
    if script["post_code"] in d:
        d[script["post_code"]]+=script["items"]  #if the "post_code" already remains in the dictionary ,then add the value
    else:
        d[script["post_code"]]=script["items"] #if it's not already in the dictionary,then make a place for it
print(list(d.items())[:10])


[('SK11 6JL', 110071), ('CW5 5NX', 38797), ('CW1 3AW', 64104), ('CW7 1AT', 43164), ('CH65 6TG', 25090), ('CH1 4DS', 34915), ('TS10 4NW', 45161), ('TS24 7PW', 58207), ('TS17 0EE', 68388), ('TS23 2DG', 31646)]


Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [34]:
items_by_post = [(k,v) for k,v in d.items()]
print(sorted(items_by_post)[:100]) # sorting the dictionary in ascending order and taking the value of first 100

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335), ('LE10 1DS'

In [35]:
postal_totals = sorted(items_by_post)[:100]

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [36]:
def group_by_post_bnf(data):
    from collections import defaultdict    
    groups=defaultdict(list)
    
    for d in data:
        groups[(d["post_code"],d["bnf_name"])].append(d["items"])  #making a dictionary with all the items same postal code and same bnf_name
        
    #let calculate the total item for each bnf_names
    
    new_group={k:sum(v) for k,v in groups.items()}
    return new_group #returns a dictionary

In [37]:
x=group_by_post_bnf(joined)
list(x.items())[:5]

[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3),
 (('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F'), 94),
 (('SK11 6JL', 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'), 9),
 (('SK11 6JL', 'Gaviscon Infant_Sach 2g (Dual Pack) S/F'), 41)]

In [38]:
total_items_by_bnf_post =[(k,v) for k,v in x.items()]
assert len(total_items_by_bnf_post) == 141196

In [39]:
total_items_by_bnf_post[:2]

[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3)]

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [40]:
total_items = {}
for k,v in x.items():
    if k[0] not in total_items:
        total_items[k[0]]=v
    else:
        total_items[k[0]]+=v
        
total_items_by_post=[(k,v) for k,v in total_items.items()]
assert len(total_items_by_post) == 118

In [41]:
total_items_by_post[:5]

[('SK11 6JL', 110071),
 ('CW5 5NX', 38797),
 ('CW1 3AW', 64104),
 ('CW7 1AT', 43164),
 ('CH65 6TG', 25090)]

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [42]:
sorted_by_max=sorted(total_items_by_bnf_post,key=lambda x : (x[0][0],-x[1],x[0][1]))  #sorting by post_code,value,bnf_name respectively
sorted_by_max[:5]

[(('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF'), 706),
 (('B11 4BW', 'Paracet_Tab 500mg'), 451),
 (('B11 4BW', 'Metformin HCl_Tab 500mg'), 387),
 (('B11 4BW', 'Lansoprazole_Cap 30mg (E/C Gran)'), 385),
 (('B11 4BW', 'Amoxicillin_Cap 500mg'), 350)]

In [43]:
max_item_by_post =[]
s=set()
for item in sorted_by_max:
    
    if item[0][0] not in s:
        max_item_by_post.append((*item[0],item[1]))     #taking the maximum value of a post. taking the values by ascending bnf_name  if more than one maximum value exists
        s.add(item[0][0])
        
len(max_item_by_post)

118

In [44]:
max_item_by_post[:10]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 706),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 556),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 1033),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 599),
 ('B70 7AW', 'Paracet_Tab 500mg', 975),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 991),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 1030),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 1869),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 1196),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 1314)]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [45]:
sorted_total_by_post=sorted(total_items_by_post)
sorted_total_by_post[:5]

[('B11 4BW', 20673),
 ('B18 7AL', 19001),
 ('B21 9RY', 29103),
 ('B23 6DJ', 24859),
 ('B70 7AW', 36531)]

In [46]:
new_lst=[]
for i in range(118):
    assert max_item_by_post[i][0]==sorted_total_by_post[i][0]
    v=max_item_by_post[i][-1]/sorted_total_by_post[i][-1]
    new_lst.append((max_item_by_post[i][0],max_item_by_post[i][1],v))
    
sorted(new_lst)[:10]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183)]

In [47]:
items_by_region = new_lst[:100]


In [48]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*